<a href="https://colab.research.google.com/github/akamalas5/Capstone/blob/main/navigating_wiki_categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract the Brake corpus from Wikipedia

In [ ]:
import pandas as pd
import re
import requests

In [ ]:
def getPages(category='Brakes'):
    S = requests.Session()

    URL = "https://en.wikipedia.org/w/api.php"

    PARAMS = {
        "action": "query",
        "cmtitle": "Category:"+category,
        "cmlimit": "200",
        #"mode": "pages",
        "list": "categorymembers",
        "depth": 5,
    #     "cmtype":"page",
        "prop": "links",
        "format": "json",
    }

    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()

    PAGES = DATA['query']['categorymembers']
    
    # traversing subcategories
    for page in PAGES:
        if(type(page) == dict and 'Category:' in page['title']):
           #{'pageid': 54336598, 'ns': 14, 'title': 'Category:Bicycle brakes'} 
           title = page['title'].split(':')[1]    
           subpages = getPages(title)
           PAGES.extend(subpages)
    return PAGES

Create dataframe from the Pages of text extracted from Wikipeida

In [ ]:
from collections import defaultdict
from tqdm import tqdm
from bs4 import BeautifulSoup

# traverse pages for given categories
page_list = getPages()
wiki_text = defaultdict(list)

for page in tqdm(page_list):
    pageid = page['pageid']
    wiki_url = 'http://en.wikipedia.org/?curid='+str(pageid)
    #print(url)
    
#     if 'brake' not in wiki_url:
#         continue
    page = requests.get(wiki_url)

    html = BeautifulSoup(page.text)
    
    paragraphs = html.find_all('p')
    
    for para in paragraphs:
#         print(para)
        if(len(para.text)>1):
            wiki_text['wiki_text'].append(para.text)
            
print('Done. Extracting table links..:', len(wiki_text['wiki_text']))
rqt_df = pd.DataFrame(wiki_text)
rqt_df.head(), len(rqt_df)

100%|██████████| 123/123 [01:05<00:00,  1.88it/s]

Done. Extracting table links..: 2877


(                                           wiki_text
 0  A brake is a mechanical device that inhibits m...
 1  Most brakes commonly use friction between two ...
 2  Brakes are generally applied to rotating axles...
 3  Since kinetic energy increases quadratically w...
 4  Almost all wheeled vehicles have a brake of so..., 2877)

Utility Function for Clean Up

In [ ]:
# function to preprocess rqt_text
def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.','',str(text))
    
    # removing new line characters
    text = re.sub('\n ','',str(text))
    # text = re.sub('\n',' ',str(text))
    
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))

    # removing any reference to outside text
    text = re.sub("[\(*?[\)\]\[]", "", str(text))
    
    # removing table tags
    text = re.sub('&nbsp;', ' ', str(text))
    text = re.sub('nbsp;', ' ', str(text))
    
    # removing table tags
    text = re.sub('  ', ' ', str(text))
    text = text.strip()
    
    return text

Create a new column with cleaned up text

In [ ]:
# preprocessing req_text
rqt_df['text_clean'] = rqt_df['wiki_text'].apply(clean)
rqt_df.loc[0, 'text_clean']

'A brake is a mechanical device that inhibits motion by absorbing energy from a moving system. It is used for slowing or stopping a moving vehicle, wheel, axle, or to prevent its motion, most often accomplished by means of friction.'

Save the Brake wiki data to a CSV file   

In [ ]:
rqt_df.to_csv('../data/wiki_brake_all_pages.csv') # -> to be sent for coref resolution